In [1]:
import numpy as np
import pandas as pd

from src.data_loader import DataLoader

In [2]:
FOURIER_SIZE = 10000

In [3]:
rows = np.array([])
ages = np.array([])
batch_size = 100
no_batches = round(DataLoader.train_size / batch_size)
for batch_no, measurements in enumerate(DataLoader.get_train_iter(batch_size=100)):
    print(f"\rBatch: {batch_no + 1:2} / {no_batches}")

    for ind, measurement in enumerate(measurements):
        print(f"\r{ind + 1:4} / {len(measurements)}", end="")
        fourier = np.fft.fft2(measurement.data.values, s=(FOURIER_SIZE, 1)).T
        # fourier = np.fft.fftshift(fourier)

        rows = np.vstack((rows, fourier)) if rows.size else fourier
        ages = np.append(ages, measurement.age)


Batch:  1 / 12
Batch:  2 / 12
Batch:  3 / 12
Batch:  4 / 12
Batch:  5 / 12
Batch:  6 / 12
Batch:  7 / 12
Batch:  8 / 12
Batch:  9 / 12
Batch: 10 / 12
Batch: 11 / 12
Batch: 12 / 12
  71 / 71

In [4]:
# Change dtype of ages to int
ages = ages.astype(int)

In [5]:
ages

array([34, 23, 51, ..., 24, 28, 68])

In [6]:
# Get modulus of complex numbers from rows
rows_with_age = np.hstack((np.abs(rows), np.angle(rows), ages.reshape(-1, 1)))
pd.DataFrame(rows_with_age).to_csv("out/fft_train.csv", index=False)

In [7]:
rows_eval = np.array([])
ages_eval = np.array([])
measurements = DataLoader.get_eval()
for ind, measurement in enumerate(measurements):
    print(f"\r{ind + 1:4} / {len(measurements)}", end="")
    fourier = np.fft.fft2(measurement.data.values, s=(FOURIER_SIZE, 1)).T

    rows_eval = np.vstack((rows_eval, fourier)) if rows_eval.size else fourier
    ages_eval = np.append(ages_eval, measurement.age)

 126 / 126

In [8]:
ages_eval = ages_eval.astype(int)


In [9]:
ages_eval

array([69, 80, 78, 42, 34, 60, 28, 28, 47, 20, 40, 47, 89, 40, 26, 30, 30,
       45, 80, 81, 59, 62, 37, 67, 67, 55, 22, 48, 44, 26, 47, 42, 54, 53,
       39, 54, 39, 38, 51, 50, 31, 37, 23, 62, 52, 44, 88, 62, 24, 49, 50,
       38, 41, 39, 24, 63, 30, 47, 64, 35, 32, 19, 25, 44, 25, 65, 68, 43,
       46, 18, 19, 31, 58, 31, 33, 54, 74, 23, 62, 25, 58, 53, 38, 28, 49,
       24, 59, 37, 68, 26, 83, 32, 66, 35, 35, 47, 26, 43, 41, 70, 19, 23,
       25, 34, 65, 79, 56, 58, 31, 48, 58, 43, 61, 29, 48, 71, 62, 43, 36,
       36, 55, 52, 81, 22, 21, 66])

In [10]:
rows_with_age_eval = np.hstack((np.abs(rows_eval), np.angle(rows_eval), ages_eval.reshape(-1, 1)))
pd.DataFrame(rows_with_age_eval).to_csv("out/fft_eval.csv", index=False)

In [11]:
pd.read_csv("out/fft_eval.csv")

,0,1,2,3,4,5,6,7,8,9,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
0,33432.835,1.150260e+04,7.297260e+04,3.792325e+04,6.640307e+04,1.333194e+04,4.621863e+04,1.741970e+04,2.398574e+04,2.446797e+04,...,-2.345484,1.177109,1.586155,-1.929709,-0.300374,2.643497,-1.147035,-1.144395,-1.530694,69.0
1,254979.770,2.113966e+05,2.809236e+05,3.519943e+05,4.902266e+05,5.355317e+05,3.487332e+05,1.694588e+05,3.680248e+05,3.115782e+05,...,-1.669310,-1.398602,-2.134801,2.596212,2.601711,3.118636,-2.153007,-1.747285,-2.560964,80.0
2,15400.125,3.397030e+05,3.008091e+05,2.384096e+05,1.297168e+05,1.243610e+05,1.049099e+05,9.252777e+04,5.441219e+04,4.637043e+04,...,-0.247698,-0.174685,-0.355534,-0.559006,-0.848600,-1.119504,-1.746505,-2.120745,-2.992718,78.0
3,917110.202,1.306606e+06,1.741195e+06,1.456121e+06,7.971880e+05,1.149658e+06,1.566820e+06,1.570458e+06,9.482254e+05,8.055031e+05,...,1.647549,-0.506296,-1.908607,-3.081234,1.807931,-0.198738,-1.978951,-3.127606,1.902379,42.0
4,148377.667,1.918916e+05,1.241636e+05,1.297377e+05,1.541850e+05,3.320227e+04,1.593026e+05,1.620211e+05,8.984985e+04,1.493286e+05,...,-1.519539,2.483133,-0.302237,-2.096255,2.196005,-1.368755,-2.517473,-1.893462,-2.496348,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,364648.520,9.796218e+04,2.189473e+05,3.419762e+05,3.544952e+05,1.159002e+05,2.266270e+05,2.372286e+05,2.331204e+05,2.049790e+05,...,0.869010,-0.043994,1.308416,0.329219,1.028661,-0.112937,0.761109,0.800438,-0.198185,52.0
122,36209.243,1.457268e+04,4.341752e+04,5.282227e+04,4.261251e+04,2.557934e+04,3.956005e+04,4.293811e+04,2.298714e+04,2.749240e+04,...,0.704291,-2.860867,0.053079,2.424967,-1.703207,1.268119,-2.631429,-0.166122,1.485323,81.0
123,1040337.680,1.552337e+06,1.287560e+06,1.131594e+06,1.490839e+06,1.012406e+06,1.569128e+06,1.235680e+06,1.098716e+06,1.416356e+06,...,-2.280479,-0.411586,2.596012,-1.732987,0.702100,-3.111146,-1.030986,1.823198,-2.479219,22.0
124,36379.846,3.547624e+04,2.253784e+04,2.878155e+04,3.319985e+04,3.790411e+04,2.986345e+04,3.165595e+04,2.752025e+04,4.672626e+03,...,-2.990449,-2.684405,-2.219546,3.009107,0.205349,-1.669771,1.938105,1.499791,0.369277,21.0
